In [1]:
import pickle
import librosa
import numpy as np

In [2]:
# load the model 
filename = 'trained_model.sav'
load_model =pickle.load(open(filename, 'rb'))
load_model.weights

Using TensorFlow backend.







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


[<tf.Variable 'conv2d_7/kernel:0' shape=(5, 5, 1, 24) dtype=float32_ref>,
 <tf.Variable 'conv2d_7/bias:0' shape=(24,) dtype=float32_ref>,
 <tf.Variable 'conv2d_8/kernel:0' shape=(5, 5, 24, 48) dtype=float32_ref>,
 <tf.Variable 'conv2d_8/bias:0' shape=(48,) dtype=float32_ref>,
 <tf.Variable 'conv2d_9/kernel:0' shape=(5, 5, 48, 48) dtype=float32_ref>,
 <tf.Variable 'conv2d_9/bias:0' shape=(48,) dtype=float32_ref>,
 <tf.Variable 'dense_5/kernel:0' shape=(2400, 64) dtype=float32_ref>,
 <tf.Variable 'dense_5/bias:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'dense_6/kernel:0' shape=(64, 5) dtype=float32_ref>,
 <tf.Variable 'dense_6/bias:0' shape=(5,) dtype=float32_ref>]

In [3]:
# Examples 
y, sr = librosa.load('Datasets/Drinking/474035__petebarry__fx-sipping-liquid-01.wav', duration=2.97)
ps1 = librosa.feature.melspectrogram(y=y, sr=sr)

y, sr = librosa.load('Datasets/Shaking/1–3 Gloucester Road 109.m4a', duration=2.97)
ps2 = librosa.feature.melspectrogram(y=y, sr=sr)

y, sr = librosa.load('Datasets/Pouring/79188__nathan-lomeli__gurgling-drinking-fountain.wav', duration=2.97)
ps3 = librosa.feature.melspectrogram(y=y, sr=sr)


In [4]:
D = [] # Dataset

D.append( (ps1, 1) ) # choose random number, which is used for validation 
D.append( (ps2, 4) )
D.append( (ps3, 3) )

In [5]:
X_test, y_test = zip(*D)
X_test = np.array([x.reshape( (128, 128, 1) ) for x in X_test])

In [6]:
# load the encoder
with open("encoderX.txt", "rb") as fp:   # Unpickling
    encoderX = pickle.load(fp)

encoderX

[['Cheers', 0],
 ['Drinking', 1],
 ['Opening _ Closing', 2],
 ['Pouring', 3],
 ['Shaking', 4]]

In [7]:
# make the predict
y_load_predict=load_model.predict(X_test)

In [8]:
y_code = y_load_predict.argmax(axis=1)
y_code_ratio = y_load_predict.max(axis=1)

y_label = []

# make the possibility<60% to 'others'
for ind, code in enumerate(y_code):
    if y_code_ratio[ind]<0.6:
        y_label.append(['others', 10])
    else:
        y_label.append(encoderX[code])
        
y_label

[['Drinking', 1], ['Shaking', 4], ['Pouring', 3]]